In [9]:
import yfinance as yf
import pandas as pd
import warnings
import numpy as np
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
import cufflinks


from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller


cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
%matplotlib inline

<font size='5'>**APPLE**</font>

In [16]:
data = yf.Ticker('aapl')
Apple_df = data.history(period='1d', start='2017-1-1', end='2022-10-1')
Apple_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-01-03,27.137807,27.262013,26.894082,27.219830,115127600,0.0,0.0
2017-01-04,27.149526,27.304199,27.126091,27.189365,84472400,0.0,0.0
2017-01-05,27.165936,27.386226,27.140157,27.327639,88774400,0.0,0.0
2017-01-06,27.367478,27.690883,27.294830,27.632296,127007600,0.0,0.0
2017-01-09,27.641664,27.988504,27.639322,27.885389,134247600,0.0,0.0


Для анализа временного ряда выберем в этом датасете колонку Volume. 
Посмотрим на график "сырых" данных.

In [29]:
Apple_df.Volume.iplot()

Применим скользящую среднюю для сглаживания данных и возможного определения каки-то паттернов. 

Как видим, при скользящей средней с окном 7 или 30 дней каких-то заметных паттернов нет.

In [86]:
Apple_df['Volume_7mean'] = Apple_df.Volume.rolling(7).mean()
Apple_df['Volume_30mean'] = Apple_df.Volume.rolling(30).mean()
Apple_df[['Volume', 'Volume_7mean', 'Volume_30mean']].iplot()

Попробуем применить двойное экспоненциальное сглаживание

In [89]:
def double_exponential_smoothing(series, alpha, beta):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    # first value is same as series
    result = [series[0]]
    for n in range(1, len(series)):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

На графике ниже заметны некоторые циклические колебание, однако несущественные.

In [90]:
Apple_df['Volume_2exp'] = double_exponential_smoothing(Apple_df.Volume, .01, .5)
Apple_df[['Volume', 'Volume_2exp']].iplot()

По крафику "сырых" и предобработанных данных видим что ряд может быть стационарным, не наблюдается заметного тренда или сезонности. 

Проведем тест Дики-Фуллера и выведем его результат. Также рассчитаем и выведем графики автокорреляции.

In [91]:
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

In [92]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,42])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

In [93]:
def test_and_plots(series):
    test_stationarity(series)
    create_corr_plot(series)
    create_corr_plot(series, plot_pacf=True)

In [94]:
test_and_plots(Apple_df.Volume)

Results of Dickey-Fuller Test:
Test Statistic                -5.943357e+00
p-value                        2.233803e-07
#Lags Used                     1.000000e+01
Number of Observations Used    1.436000e+03
Critical Value (1%)           -3.434912e+00
Critical Value (5%)           -2.863555e+00
Critical Value (10%)          -2.567843e+00
dtype: float64


В результате теста получаем значение p-value существенно меньше 0.05, на основании этого мы отклоняем гипотезу о нестационарности ряда и делаем вывод, что он стационарен. На основании такого ряда можно строить прогнозы.

<font size='5'>**GOOGLE**</font>

In [96]:
data = yf.Ticker('goog')
Google_df = data.history(period='1d', start='2017-1-1', end='2022-10-1')
Google_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-01-03,38.940498,39.481499,38.790001,39.306999,33146000,0,0.0
2017-01-04,39.417999,39.567001,39.158001,39.345001,21460000,0,0.0
2017-01-05,39.304001,39.723999,39.250999,39.701000,26704000,0,0.0
2017-01-06,39.763000,40.395000,39.610199,40.307499,32804000,0,0.0
2017-01-09,40.320000,40.498299,40.141499,40.332500,25492000,0,0.0


 Для разнообразоя в этом датасете для анализа возьмем колонку High. Тем более мы понимаем, что это значение чуть другой природы и по другому ведет себя во времени.

In [98]:
Google_df.High.iplot()

Проводим все те же манипуляции со значениями временного ряда. 

Считаем скользящие средние.
Как видим на графиках ниже, существенно новой информации о данных это нам не дало. Но все также заметно наличие тренда.

In [108]:
Google_df['High_7mean'] = Google_df.High.rolling(7).mean()
Google_df['High_30mean'] = Google_df.High.rolling(30).mean()
Google_df[['High', 'High_7mean', 'High_30mean']].iplot()

In [109]:
Google_df['High_2exp'] = double_exponential_smoothing(Google_df.High, .01, .5)
Google_df[['High', 'High_2exp']].iplot()

Проведем тест Дики-Фуллера на стационарность временного ряда и построим графики автокорреляции и частичной автокорелляции.

In [111]:
test_and_plots(Google_df.High)

Results of Dickey-Fuller Test:
Test Statistic                   -1.086790
p-value                           0.720311
#Lags Used                        4.000000
Number of Observations Used    1442.000000
Critical Value (1%)              -3.434893
Critical Value (5%)              -2.863546
Critical Value (10%)             -2.567838
dtype: float64


Как видим по результатам теста имеем достаточно высокое значение p-value 0.72, а следовательно отклонить гипотезу о нестационарности мы не можем. Будем пытаться избавиться от нестационарности.

Первым делом пробуем применить преобразование Бокса-Кокса и оценить временной ряд на стационарность еще раз.

In [115]:
from scipy.stats import boxcox

Google_df['High_BC'] = boxcox(Google_df.High, 0)
Google_df['High_BC'].iplot()
test_and_plots(Google_df.High_BC)

Results of Dickey-Fuller Test:
Test Statistic                   -1.392438
p-value                           0.585896
#Lags Used                        1.000000
Number of Observations Used    1445.000000
Critical Value (1%)              -3.434884
Critical Value (5%)              -2.863542
Critical Value (10%)             -2.567836
dtype: float64


Все еще наблюдается высокое значение p-value 0.59.

Попробуем продифференцировать временной ряд и после этого оценить его стационарность.

In [128]:
Google_df['1stdiff'] = Google_df.High.diff()
Google_df['1stdiff'][0] = 0
Google_df['1stdiff'].iplot()
test_and_plots(Google_df['1stdiff'])

Results of Dickey-Fuller Test:
Test Statistic                  -20.641085
p-value                           0.000000
#Lags Used                        3.000000
Number of Observations Used    1443.000000
Critical Value (1%)              -3.434890
Critical Value (5%)              -2.863545
Critical Value (10%)             -2.567837
dtype: float64


Как видим значение p-value стало равно 0, значит ряд стационарен.